# Importando os dados

In [ ]:
# Instala as dependências do projeto
!pip install -r ../../../requirements.txt

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
dados = pd.read_csv('../../../data/dados_cvm.csv')

### Nessa etapa atribuímos o VR, e também o target, mas sem fazer a divisão pelo patrimônio líqudo
- Optamos por não fazer a divisão pelo patrimônio líquido para análisar as hipóteses, pois tem muitos PL zerados ou inconsistentes, afetando a veracidade dos dados.

In [ ]:
dados = dados.assign(vr_sem_riscos_e_beneficios = dados['Ativo_Direitos_Sem_Aquisicao_Parcelas_Inadimplentes']  + dados['Ativo_Direitos_Sem_Aquisicao_Creditos_Inadimplentes'] - dados['Carteira_Direitos_Sem_Aquisicao_Inadimplentes_1_30_Dias'] - dados['Ativo_Direitos_Sem_Aquisicao_Provisao_Reducao'])
dados  = dados.assign(vr_com_riscos_e_beneficios = dados['Ativo_Direitos_Aquisicao_Parcelas_Inadimplentes']  + dados['Ativo_Direitos_Aquisicao_Creditos_Inadimplentes'] - dados['Carteira_Direitos_Aquisicao_Inadimplentes_1_30_Dias'] - dados['Ativo_Direitos_Aquisicao_Provisao_Reducao'])
dados  = dados.assign(target = round((dados['vr_sem_riscos_e_beneficios'] + dados['vr_com_riscos_e_beneficios'])))

# Fazendo análise de Inadimplência vs Provisão

## Análise Geral
- Primeiro vou plotar um gráfico somando todas as provisões e inadimplências dos fundos em determinado mês para analisar o comportamento comum.

-  *Agg é agregate, serve para aplicar uma ou mais funções de agregação em um grupo de dados após serem agrupados por uma chave*

In [ ]:
grouped_data = dados.groupby('Data_Competencia').agg({
    'Ativo_Direitos_Aquisicao_Creditos_Inadimplentes': 'sum',
    'Ativo_Direitos_Aquisicao_Provisao_Reducao': 'sum'
}).reset_index()

# Plotando o gráfico
plt.figure(figsize=(10, 6))
plt.plot(grouped_data['Data_Competencia'], grouped_data['Ativo_Direitos_Aquisicao_Creditos_Inadimplentes'], color='red', label='Inadimplências')
plt.plot(grouped_data['Data_Competencia'], grouped_data['Ativo_Direitos_Aquisicao_Provisao_Reducao'], color='green', label='Provisões')
plt.xlabel('Data Competência')
plt.ylabel('Valor')
plt.title('Inadimplências vs. Provisões por Data Competência')
plt.legend()
plt.grid(True)
plt.show()


## Separando os Anormais
- Nessa próxima etapa eu faço a mesma soma de creditos inadimplentes e provisões, só que eu filtro apenas os fundos que tiveram uma soma de inadimplência maior do que de provisão

In [ ]:
grouped_data = dados.groupby('ID_Participante').agg({
    'Ativo_Direitos_Aquisicao_Creditos_Inadimplentes': 'sum',
    'Ativo_Direitos_Aquisicao_Provisao_Reducao': 'sum'
}).reset_index()

# Filtrando os dados onde a soma das provisões é menor que a soma das inadimplências
filtered_data = grouped_data[grouped_data['Ativo_Direitos_Aquisicao_Provisao_Reducao'] < grouped_data['Ativo_Direitos_Aquisicao_Creditos_Inadimplentes']]

In [ ]:
filtered_data['ID_Participante']

In [ ]:
# Lista de IDs com provisões menores que inadimplencia
ids_to_keep = filtered_data['ID_Participante']

dados_inadimplentes = dados[dados['ID_Participante'].isin(ids_to_keep)]

dados_inadimplentes.shape

### Plotando o gráfico
- Aqui, já com os fundos anômalos filtrados eu faço o mesmo método de somar todas as inadimplências e provisões do Mês e plotar a diferença

In [ ]:
grouped_data = dados_inadimplentes.groupby('Data_Competencia').agg({
    'Ativo_Direitos_Aquisicao_Creditos_Inadimplentes': 'sum',
    'Ativo_Direitos_Aquisicao_Provisao_Reducao': 'sum'
}).reset_index()

plt.figure(figsize=(10, 6))
plt.plot(grouped_data['Data_Competencia'], grouped_data['Ativo_Direitos_Aquisicao_Creditos_Inadimplentes'], color='red', label='Inadimplências')
plt.plot(grouped_data['Data_Competencia'], grouped_data['Ativo_Direitos_Aquisicao_Provisao_Reducao'], color='green', label='Provisões')
plt.xlabel('Data Competência')
plt.ylabel('Valor')
plt.title('Inadimplências vs. Provisões por Data Competência - Provisões < Inadimplências')
plt.legend()
plt.grid(True)
plt.show()

- A conclusão é que muitos fundos não tem uma provisão suficiente para cobrir a inadimplência, e isso é um problema para a saúde financeira do fundo.

# Hipóteses do TAPI propostas pela CVM

## Hipótese 1
- FIDCs mais concentrados em poucos cedentes (campos I.a.12 e I.b.12) tendem a ter aumento de VR ao longo 
dos meses? Qual o nível de concentração de cedentes antecede um aumento do VR? 
A soma dos percentuais relativos aos cedentes, informados ou não no campo I.a.12, não necessariamente 
somam 100%, tampouco os do campo I.b.12. 

### Analisando número de cedentes x VR

In [ ]:
dados_cedentes = pd.read_csv('../data/dados_cvm_cedentes.csv')

dados_cedentes.drop(['Item', 'CPF_CNPJ_Cedente'], axis=1, inplace=True)

dados_v1 = pd.merge(dados, dados_cedentes, on="SK_Documento", how="inner")

In [ ]:
len(dados_cedentes)

In [ ]:
dados_v1.shape

In [ ]:
dados_v1['Participacao_Percentual'].isnull().sum()

In [ ]:
dados_v1['Participacao_Percentual'].value_counts()

In [ ]:
dados_filtrados = dados_v1[dados_v1['Participacao_Percentual'] == 100.00]

In [ ]:
media_filtrados = dados_filtrados['target'].mean()
media_total = dados['target'].mean()

dados_medias = [media_total, media_filtrados]
labels = ['Geral', 'Com apenas 1 Cedente']

plt.bar(labels, dados_medias, color='red')  
plt.ylabel('Média do target')
plt.title('Comparação das Médias do target')
plt.show()

#### Hipótese comprovada
- Apesar de terem 22798 dados zerados o que dificulta a análise, é possível já extrair informações relevantes dos fundos em que só tem 1 cedente com 100% de participação.
- Analisando os fundos com apenas 1 cedente é bem perceptível que o target é muito superior.

## Hipótese 2
- Fundos com recompras de créditos (3 campos sob o item VII.d) são mais propensos a apresentarem aumento 
futuro de VR? Qual o nível de recompras antecede um aumento do VR? 

In [ ]:
dados[['Negocios_Recompras_Quantidade', 'Negocios_Recompras_Valor', 'Negocios_Recompras_Valor_Contabil']].describe()


In [ ]:
colunas_interesse = ['Negocios_Recompras_Quantidade', 'Negocios_Recompras_Valor', 'Negocios_Recompras_Valor_Contabil', 'target']
dados_selecionados = dados[colunas_interesse]

matriz_correlacao = dados_selecionados.corr()

plt.figure(figsize=(5, 4))
sns.heatmap(matriz_correlacao, annot=True, cmap='coolwarm', center=0)
plt.title('Fundos Gerais - Matriz de Correlação')
plt.show()

- Plotando os dados separados como inadimplentes
    - Esses dados são aqueles filtrados antes que tinham inadimplências maiores que provisões

In [ ]:
dados_inadimplentes[['Negocios_Recompras_Quantidade', 'Negocios_Recompras_Valor', 'Negocios_Recompras_Valor_Contabil']].describe()


In [ ]:
colunas_interesse = ['Negocios_Recompras_Quantidade', 'Negocios_Recompras_Valor', 'Negocios_Recompras_Valor_Contabil', 'target']
dados_selecionados = dados_inadimplentes[colunas_interesse]

matriz_correlacao = dados_selecionados.corr()

plt.figure(figsize=(5, 4))
sns.heatmap(matriz_correlacao, annot=True, cmap='coolwarm', center=0)
plt.title('Fundos com mais Inadimplências')
plt.show()

#### Hipótese descomprovada a princípio pela nossa análise
- O target está correlacionado negativamente com os campos sugeridos na hipótese para os fundos que tem menos provisões que inadimplências e apresenta uma correlação muito baixa nos fundos gerais.

## Hipótese 3
- Fundos com substituições de créditos (3 campos sob o item VII.c) são mais propensos a apresentar aumento 
futuro de VR? Qual o nível de substituições antecede um aumento do VR?

In [ ]:
dados[['Negocios_Substituicoes_Quantidade', 'Negocios_Substituicoes_Valor', 'Negocios_Substituicoes_Valor_Contabil']].describe()


In [ ]:
colunas_interesse = ['Negocios_Substituicoes_Quantidade', 'Negocios_Substituicoes_Valor', 'Negocios_Substituicoes_Valor_Contabil', 'target']
dados_selecionados = dados[colunas_interesse]

matriz_correlacao = dados_selecionados.corr()

plt.figure(figsize=(5, 4))
sns.heatmap(matriz_correlacao, annot=True, cmap='coolwarm', center=0)
plt.title('Fundos Gerais - Matriz de Correlação')
plt.show()

- Fazendo a análise com os dados separados como inadimplentes

In [ ]:
dados_inadimplentes[['Negocios_Substituicoes_Quantidade', 'Negocios_Substituicoes_Valor', 'Negocios_Substituicoes_Valor_Contabil']].describe()

In [ ]:
colunas_interesse = ['Negocios_Substituicoes_Quantidade', 'Negocios_Substituicoes_Valor', 'Negocios_Substituicoes_Valor_Contabil', 'target']
dados_selecionados = dados_inadimplentes[colunas_interesse]

matriz_correlacao = dados_selecionados.corr()

plt.figure(figsize=(5, 4))
sns.heatmap(matriz_correlacao, annot=True, cmap='coolwarm', center=0)
plt.title('Fundos com mais Inadimplências')
plt.show()

#### Hipótese possívelmente comprovada a princípio pela nossa análise
- O target está correlacionado positivamente com os campos de substituição nos fundos inadimplentes, apesar de serem valores baixos, com um possível refinamento junto a modelagem será possível chegar a uma resposta mais precisa, devido a complexidade dos dados.
- Contudo ele está correlacionado negativamente nos fundos generalizados, o que ainda pode causar uma imprecisão na análise

## Hipótese 4
- FIDCs com renegociações de créditos são mais propensos a apresentar aumento futuro de VR? Qual o nível 
de renegociações antecede um aumento do VR? 
As renegociações são identificadas por um aumento de créditos a vencer (campos I.2a.1 e I.2b.1) e 
diminuição dos valores em atraso (campos I.2a.2.1 e I.2b.2.1 ou V.b e VI.b) no mesmo mês, sem a ocorrência 
de recompra nem substituição de créditos. 

In [ ]:
dados["Data_Competencia"] = pd.to_datetime(dados["Data_Competencia"], format="%Y-%m-%d")

Quando passa um mês na 'Data-Competencia' que já está no formato %Y-%m-%d

- Aumenta 'Ativo_Direitos_Aquisicao_Creditos_Vencer_Adimplentes'
- Diminui 'Carteira_Direitos_Aquisicao_Inadimplentes' + 'Carteira_Direitos_Sem_Aquisicao_Prazo'

In [ ]:
def filter_and_compare(df):
    df.sort_values(['ID_Participante', 'Data_Competencia'], inplace=True)
    
    df['Diff_Ativo'] = df.groupby('ID_Participante')['Ativo_Direitos_Aquisicao_Creditos_Vencer_Adimplentes'].diff()
    df['Diff_Carteira'] = df.groupby('ID_Participante')['Carteira_Direitos_Aquisicao_Inadimplentes'].shift(1) + df.groupby('ID_Participante')['Carteira_Direitos_Sem_Aquisicao_Prazo'].shift(1)
    df['Diff_Carteira'] = df.groupby('ID_Participante')['Diff_Carteira'].diff()

    filtered_df = df[(df['Diff_Ativo'] > 0) & (df['Diff_Carteira'] < 0)]
    
    return filtered_df


filtered_result = filter_and_compare(dados)

- Filtrando os dados em que não ocorre recompra / substituição de crédito
    - 'Negocios_Substituicoes_Quantidade' = 0
    - Negocios_Recompras_Quantidade = 0


In [ ]:
dados_sem_substituicao_recompra = filtered_result[(filtered_result['Negocios_Substituicoes_Quantidade'] == 0) & (filtered_result['Negocios_Recompras_Quantidade'] == 0)]

In [ ]:
colunas_interesse = ['Diff_Ativo', 'Diff_Carteira', 'target']
dados_selecionados = dados_sem_substituicao_recompra[colunas_interesse]

matriz_correlacao = dados_selecionados.corr()

plt.figure(figsize=(5, 4))
sns.heatmap(matriz_correlacao, annot=True, cmap='coolwarm', center=0)
plt.title('Matriz de Correlação')
plt.show()

#### Hipótese possívelmente comprovada a princípio pela nossa análise
- O target está correlacionado sim com os campos dessa hipótese, mas não é uma correlação forte o suficiente para comprovar ela definitivamente. Após uma análise mais precisa e a modelagem será possível confirmar o impacto das negociações ao longo do tempo, talvez apresentando uma correlação maior, mas já nesse estado inicial é possível ver que a hipótese é possívelmente verdadeira.

## Hipótese 5
- Algum(ns) ramo(s) de atividade econômica do FIDC é(são) mais propenso(s) a registrar maior aumento no 
VR? 
A Tabela II é útil para essa análise, porém não traz a divisão de DCs entre com e sem riscos e benefícios.

In [ ]:
colunas_interesse = ['Carteira_Industrial', 'Carteira_Mercado_Imobiliario','Carteira_Comercial_Total', 'Carteira_Arrendamento_Mercantil', 'Carteira_Servicos_Total', 'Carteira_Entretenimento', 'Carteira_Agronegocio', 'Carteira_Financeiro', 'Carteira_Credito Pessoal', 'Carteira_Credito_Pessoal_Consignado', 'Carteira_Credito_Corporativo', 'Carteira_Middle_Market', 'Carteira_Veiculos', 'Carteira_Imobiliaria_Empresarial', 'Carteira_Imobiliaria_Residencial', 'Carteira_Outros_Financeiro', 'Carteira_Cartao_Credito', 'Carteira_Factoring', 'Carteira_Factoring_Pessoal', 'Carteira_Factoring_Corporativo', 'Carteira_Setor_Publico', 'Carteira_Precatorios', 'Carteira_Creditos_Tributarios', 'Carteira_Royalties', 'Carteira_Outros_Setor_Publico', 'Carteira_Acoes_Judiciais', 'Carteira_Propriedade_Intelectual',  'target']
dados_selecionados = dados[colunas_interesse]

correlacoes = dados_selecionados.corr()['target']


correlacoes_df = pd.DataFrame(correlacoes)
correlacoes_df = correlacoes_df.sort_values(by='target', ascending=True)
correlacoes_df = correlacoes_df.drop('target')
correlacoes_df.columns = ['Correlação com Target']

plt.figure(figsize=(10, 6))
sns.barplot(x=correlacoes_df.index, y='Correlação com Target', data=correlacoes_df)
plt.title('Correlação do Target com Outras Colunas')
plt.xticks(rotation=90)
plt.show()

- Vendo com os dados inadimplentes

In [ ]:
dados_selecionados = dados_inadimplentes[colunas_interesse]

correlacoes = dados_selecionados.corr()['target']


correlacoes_df = pd.DataFrame(correlacoes)
correlacoes_df = correlacoes_df.sort_values(by='target', ascending=True)
correlacoes_df = correlacoes_df.drop('target')
correlacoes_df.columns = ['Correlação com Target']

plt.figure(figsize=(10, 6))
sns.barplot(x=correlacoes_df.index, y='Correlação com Target', data=correlacoes_df)
plt.title('Correlação do Target com Outras Colunas')
plt.xticks(rotation=90)
plt.show()

- Fazendo Teste com target > 1 

In [ ]:
dados_t_positivo = dados[dados['target'] > 0]
dados_selecionados = dados_t_positivo[colunas_interesse]

correlacoes = dados_selecionados.corr()['target']


correlacoes_df = pd.DataFrame(correlacoes)
correlacoes_df = correlacoes_df.sort_values(by='target', ascending=True)
correlacoes_df = correlacoes_df.drop('target')
correlacoes_df.columns = ['Correlação com Target']

plt.figure(figsize=(10, 6))
sns.barplot(x=correlacoes_df.index, y='Correlação com Target', data=correlacoes_df)
plt.title('Correlação do Target com Outras Colunas')
plt.xticks(rotation=90)
plt.show()

#### Hipótese comprovada
- Essa hipótese foi comprovada, pois dependendo da carteira de crédito do fundo, ele tem uma maior chance de ter um aumento no target. Selecionando os fundos com target acima de um é perceptível que a carteira financeira e o crédito pessoal consignado tem grande correlação com o target. Portanto, o tipo de carteira predominante em um fundo pode ser um fator determinante para o aumento do target.

## Hipótese 6
- De forma geral, DCs com e sem riscos e benefícios comportam-se forma diferente ou não, quanto à evolução 
da inadimplência dos seus créditos e do VR? 

In [ ]:
colunas_interesse = ['Ativo_Direitos_Aquisicao_Creditos_Inadimplentes', 'vr_com_riscos_e_beneficios', 'vr_sem_riscos_e_beneficios', 'target']
dados_selecionados = dados[colunas_interesse]

matriz_correlacao = dados_selecionados.corr()

plt.figure(figsize=(5, 4))
sns.heatmap(matriz_correlacao, annot=True, cmap='coolwarm', center=0)
plt.title('Matriz de Correlação')
plt.show()

#### Essa hipótese se comprova
- Tanto por essa análise quanto a nossa exploratória dos dados, é perceptível que o vr com riscos e benefícios tem uma maior correlação com o target, ou seja, ele influencia mais no target do que o vr sem riscos e benefícios.